## CV2 Read batch

This notebook is used to detect faces from pictures in batch. It consists of three functions:
- Detect faces based on haarcascade
- Detect faces based on opencv deep learning
- Detect faces combining haarcascade and deep learning

Besides it has two possibilities to process images:
- Read images from a specific folder, process them and store the processed image in another folder
- Crawl over a directory structure and make the same structure for processed images


In [23]:
# import the necessary packages
import os
import numpy as np
import argparse
import cv2
import glob
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Initialize the haarcascade classifier for face detection
cascade_file_src = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascade_file_src)

In [3]:
# Some command line arguments are needed to be able to execute the deep learning model for face detection
args = {
    "prototxt": "deploy.prototxt.txt",
    "model": "res10_300x300_ssd_iter_140000.caffemodel",
    }

{'model': 'res10_300x300_ssd_iter_140000.caffemodel',
 'prototxt': 'deploy.prototxt.txt'}

In [4]:
# load the serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


## Deep learning function

In [15]:
def ext_faces(image):
    # Determine dimensions of picture
    (h, w) = image.shape[:2]
    # Prepare the image for the network
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()
    faces =[]
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the `confidence` is greater than the minimum confidence.
        if confidence > 0.9: 
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            #print(box)
            (startX, startY, endX, endY) = box.astype("int")
            img = cv2.rectangle(image,(startX,startY),(endX,endY),(255,0,0),2)
            img = image[startY:endY, startX:endX]
            face = cv2.resize(img,((224,224)))
            faces.append(face)
    print('number of faces found:', len(faces))
    return(faces)
            

## Haarcascade function

In [10]:
def ext_faces2(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # detect faces
    # scaleFactor and minNeighbours are set high
    # Clarification properties: http://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_Object_Detection_Face_Detection_Haar_Cascade_Classifiers.php
    faces = faceCascade.detectMultiScale(gray, 1.3, 5)
    print('# faces found:',len(faces))
    facecrop = []
    # crop right area
    for f in faces:
            x, y, w, h = [ v for v in f ]
            facecrop.append(image[y:y+h, x:x+w])
    facecrop2 = []
    # resize to right number of pixels
    for face in facecrop:
        crpim = cv2.resize(face,((224,224)))
        facecrop2.append(crpim)
    return facecrop2

## Function that searches faces with haarcascade and confirms with deep learning

In [21]:
def ext_faces3(image):
    # First haarcascade is used to find faces 
    # Using low values of scaleFactor and minNeighbours (with many false positives)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.2, 3)
    facecrop = []
    # The faces are cropped and the resulting images are fed in a sequence to the neural net.
    for f in faces:
        x, y, w, h = [ v for v in f ]
        facecrop.append(image[y:y+h, x:x+w])
    confirmed = []
    for img in facecrop:    
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        # Probability function of 0.9 can be changed to filter false positives
        if detections[0, 0, 0, 2] > 0.9:
            face = cv2.resize(img,((224,224)))
            confirmed.append(face)
    print('# faces found:',len(confirmed))
    return(confirmed)
            

## Processing files in one folder

In [22]:
cv_img = []
names =[]
for img in glob.glob("nofaces/*.jpg"):
    image= cv2.imread(img)
    name = img[8:-4]
    print(name)
    faces = ext_faces3(image)
    for i, face in enumerate(faces):
        cv2.imwrite("output/" + name + "_" + str(i) + ".jpg", face)


20151003_085614
# faces found: 0
20151003_085629
# faces found: 0
20151003_085633
# faces found: 0
20151003_085641
# faces found: 0
20151003_085643
# faces found: 0
20151003_085648
# faces found: 0
20151003_085723
# faces found: 0
20151003_085725
# faces found: 0
20151003_085734
# faces found: 0
20151003_085827
# faces found: 0
20151003_085948
# faces found: 0
20151003_085950
# faces found: 0
IMG-20151003-WA0029
# faces found: 0


## Processing files in multiple folders

In [28]:
# Iterate over directories and files
# Creates equal directory structure with processed files
# Faces in the wild: http://vis-www.cs.umass.edu/lfw/index.html

rootdir ='lfw/'
newdir = 'output'

for subdir, dirs, files in os.walk(rootdir):
    crdir = newdir + subdir[4:]
    print(crdir)
    os.makedirs(crdir)
    for file in files:
        if file[-4:] == ".jpg":
            filepath = subdir + '/' + file
            #print(filepath)
            image= cv2.imread(filepath)
            name = file[:-4]
            faces = ext_faces3(image)
            for i, face in enumerate(faces):
                wrfile = newdir + subdir[4:] + '/' + name + "_" + str(i) + ".jpg"
                print(wrfile)
                cv2.imwrite(wrfile, face)

output6/
output6/.ipynb_checkpoints
output6/Bill_Simon
# faces found: 1
output6/Bill_Simon/Bill_Simon_0001_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0002_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0003_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0004_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0005_0.jpg
# faces found: 0
# faces found: 1
output6/Bill_Simon/Bill_Simon_0007_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0008_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0009_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0010_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0011_0.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0012_0.jpg
# faces found: 2
output6/Bill_Simon/Bill_Simon_0013_0.jpg
output6/Bill_Simon/Bill_Simon_0013_1.jpg
# faces found: 1
output6/Bill_Simon/Bill_Simon_0014_0.jpg
# faces found: 2
output6/Bill_Simon/Bill_Simon_0015_0.jpg
output6/Bill_Simon/Bill_Simon_0015_1.jpg
output6/Britney_Spears
# faces fou

# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0006_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0007_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0008_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0009_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0010_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0011_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0012_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0013_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0014_0.jpg
# faces found: 1
output6/Meryl_Streep/Meryl_Streep_0015_0.jpg
output6/Nancy_Pelosi
# faces found: 1
output6/Nancy_Pelosi/Nancy_Pelosi_0001_0.jpg
# faces found: 1
output6/Nancy_Pelosi/Nancy_Pelosi_0002_0.jpg
# faces found: 1
output6/Nancy_Pelosi/Nancy_Pelosi_0003_0.jpg
# faces found: 1
output6/Nancy_Pelosi/Nancy_Pelosi_0004_0.jpg
# faces found: 1
output6/Nancy_Pelosi/Nancy_Pelosi_0005_0.jpg
# faces found: 1
output6/Nancy_Pelosi/Nancy_Pelos